In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Agenda, load the segmentation model, get the mask, binarize it

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import models

from skimage import io
from skimage.transform import resize
from skimage.color import rgb2gray,gray2rgb
from matplotlib import pyplot as plt

import csv
import numpy as np
import cv2

In [ ]:
## Testing simple Image read / EDA
import csv
import numpy as np

filename = "../input/hpa-single-cell-image-classification/train.csv"
fields = [] 
rows = [] 
codes = [
'Nucleoplasm',
'Nuclear membrane',
'Nucleoli',
'Nucleoli fibrillar center',
'Nuclear speckles',
'Nuclear bodies',
'Endoplasmic reticulum',
'Golgi apparatus',
'Intermediate filaments',
'Actin filaments',
'Microtubules',
'Mitotic spindle',
'Centrosome',
'Plasma membrane',
'Mitochondria',
'Aggresome',
'Cytosol',
'Vesicles and punctate cytosolic patterns',
'Negative'
]

l = [0] * 19

# reading csv file 
with open(filename, 'r') as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
    
    fields = next(csvreader)
    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row) 
    
    for row in rows[:50]: # Reading the First image and label
        image_id = row[0]
        labels = row[1]
        labels = labels.replace("|",",")
        list_labels = list(labels.split(",")) 
        
        if(len(list_labels) == 1):
            for i in list_labels:
                l[int(i)] +=1 
                
        

        img_blue = io.imread("../input/hpa-single-cell-image-classification/train/"+ image_id+"_blue.png")
        img_green = io.imread("../input/hpa-single-cell-image-classification/train/"+ image_id+"_green.png")
        img_red = io.imread("../input/hpa-single-cell-image-classification/train/"+ image_id+"_red.png")
        img_yellow = io.imread("../input/hpa-single-cell-image-classification/train/"+ image_id+"_yellow.png")
        
        #https://www.kaggle.com/thedrcat/hpa-single-cell-classification-eda/comments#Contents
        rgb = np.dstack((img_red,img_yellow,img_blue))
        gray =  cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
        ret,thresh1 = cv2.threshold(gray,10,255,cv2.THRESH_BINARY)
        ret,thresh2 = cv2.threshold(img_blue,10,255,cv2.THRESH_BINARY)
        ret,thresh3 = cv2.threshold(img_green,10,255,cv2.THRESH_BINARY)
        fig, ax = plt.subplots(1,4, figsize=(20,50))
        

        sub = thresh1 - thresh2
        
       
        
        ax[0].imshow(rgb)
        ax[0].axis('off')
        
        ax[1].imshow(thresh2,cmap="gray")
        ax[1].axis('off')
        
        ax[2].imshow(thresh3,cmap="gray")
        ax[2].axis('off')
        
        ax[3].imshow(sub,cmap="gray")
        ax[3].axis('off')
        
#         ax[3].imshow(img_blue)
#         ax[3].axis('off')
        plt.show()
        



print(l)
print(codes)